In [142]:
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

from db_queries import username, password, dsn, dbhostname, service_name, dbtables, querys
from table_functions import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def combine_final_table_pipeline(data):

    # usuwanie znaków białych z DMC[MEB_DGM] i DMC_CASTING[MEB_DMC]
    data['MEB_DMC'].dmc_casting = data['MEB_DMC']['dmc_casting'].str.strip()
    data['MEB_DGM'].dmc = data['MEB_DGM']['dmc'].str.strip()

    # usuwanie z meb_dmc wierszy z 'WORKPIECE NIO' w kodzie DMC
    data['MEB_DMC'] = data['MEB_DMC'][~data['MEB_DMC']['dmc'].str.contains('WORKPIECE', case=False, na=False)]

    # wybieranie rekordów dla MEB+ 
    data['MEB_DGM'] = data['MEB_DGM'][(data['MEB_DGM']['nr_dgm'].between(8, 10)) & (data['MEB_DGM']['dmc'].apply(lambda x: len(str(x)) == 21))]
    # usunięcie anomalii z MEB_DMC
    data['MEB_DMC'] = data['MEB_DMC'][data['MEB_DMC']['dmc'].str[:3] == '0MH']

    # przygotowywuję tabelę ONI_CIRCUITS do połączenia 
    oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['assigment', 'flow', 'set_point', 'start_delay', 'temp', 'working_mode'])
    oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
    oni_circuits.reset_index(inplace=True)

    final_table = data['MEB_DGM'].copy()
    final_table.drop(columns=['rn'], inplace=True)

    # łączę z tabelą MEB_DGM
    final_table = final_table.merge(oni_circuits, left_on='id', right_on='id_dmc', how='inner')
    final_table.drop(columns=['id_dmc_y'], inplace=True)
    final_table.rename(columns={'id_dmc_x': 'id_dmc'}, inplace=True)

    # łączę tabelę MEB_DMC z MEB_DGM
    final_table = final_table.merge(data['MEB_DMC'], left_on='dmc', right_on='dmc_casting', how='left', suffixes=('_DGM', '_DMC'))


    return final_table

# DGM id - oni id_dmc
# DGM dmc - MEB_DMC dmc_casting

def read_last_meb_dgm(last_id = 0):

    data = {}
    if last_id:
        query = f"""SELECT *
            FROM (
                SELECT
                    t.*,
                    ROW_NUMBER() OVER (PARTITION BY DMC ORDER BY ID DESC) AS rn
                FROM
                    Z3DMC.MEB_DGM t
            ) subquery
            WHERE rn = 1
            AND id > {last_id}"""

    else:
        query = """SELECT *
            FROM (
                SELECT *
                FROM Z3DMC.MEB_DGM
                ORDER BY ID DESC
            )
            WHERE ROWNUM = 1
            """
    
    try:
        sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
        engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
        data.update({'MEB_DGM': pd.read_sql(query, engine)})
    except SQLAlchemyError as e:
        print(e)
    
    data['MEB_DGM'].drop(columns=['timestamp','data_znakowania','data_odlania', 'metal_level', 'metal_pressure', 'max_press_kolbenhub', 'oni_temp_curr_f2'], inplace= True)

    last_id = data['MEB_DGM'].id.max()
    return data, last_id

def check_if_meb_base(data):
    data['MEB_DGM'].dmc = data['MEB_DGM']['dmc'].str.strip()
    data['MEB_DGM'] = data['MEB_DGM'][(data['MEB_DGM']['nr_dgm'].between(8, 10)) & (data['MEB_DGM']['dmc'].apply(lambda x: len(str(x)) == 21))]

    if data['MEB_DGM'].empty:
        print('There are not MEB_BASE+ part produced since last time')
        return 1
    else:
        return data

def read_oni(data):
    id_list = list(data['MEB_DGM'].id)
    ids_ranges = [id_list[x:x+500] for x in range(0, len(id_list), 500)]
    ids_ranges_tuples = [tuple(sublist) for sublist in ids_ranges]

    result_df = pd.DataFrame()
    try:
        sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
        engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)

        for ids in ids_ranges_tuples:
            query = f"""SELECT ID_DMC, CIRCUIT_NR, 
                    MAX(ASSIGMENT) AS ASSIGMENT, 
                    MAX(FLOW) AS FLOW, 
                    MAX(SET_POINT) AS SET_POINT,
                    MAX(START_DELAY) AS START_DELAY,
                    MAX(TEMP) AS TEMP,
                    MAX(WORKING_MODE) AS WORKING_MODE
                FROM Z3DMC.ONI_CIRCUITS
                WHERE ID_DMC IN {ids}
                GROUP BY ID_DMC, CIRCUIT_NR
                ORDER BY ID_DMC
                """
    
            df = pd.read_sql(query, engine)
            result_df = pd.concat([result_df, df], ignore_index=True)

            
    except SQLAlchemyError as e:
        print(e)

    data.update({'ONI_CIRCUITS': result_df})
    
    return data

def combine_into_one_table(data):
    oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['assigment', 'flow', 'set_point', 'start_delay', 'temp', 'working_mode'])
    oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
    oni_circuits.reset_index(inplace=True)
    final_table = data['MEB_DGM'].copy()
    final_table = final_table.merge(oni_circuits, left_on='id', right_on='id_dmc', how='inner')

    return final_table

def categorize_data_pipeline(whole_df):
    final_table = whole_df.copy()
    categorical_columns = []
    for name in ['assigment', 'working_mode']:
        for x in range(1,29):
            categorical_columns.append(f'{name}_{x}')

    categorical_data = final_table[categorical_columns].astype('category')
    categorical_data = pd.get_dummies(categorical_data, drop_first=True, dtype=int)
    
    final_table.drop(columns=categorical_columns, inplace=True)
    final_table = pd.concat([final_table, categorical_data], axis=1)

    return final_table

In [143]:
dgm, id_max = read_last_meb_dgm(1474000)
dgm = check_if_meb_base(dgm)
dgm_oni = read_oni(dgm)


C:\Users\DLXPMX8\AppData\Local\Temp\ipykernel_20104\64739301.py:119: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df], ignore_index=True)


In [131]:
final_tab = combine_into_one_table(dgm_oni)

In [144]:
final_tab = categorize_data_pipeline(final_tab)

In [147]:
train_data = load_csv('final_table_before_normalization.csv')

File to read:
c:/Users/DLXPMX8/Desktop/Projekt_AI/meb_process_data_analysis/src/.data/final_table_before_normalization.csv


In [151]:
columns_to_drop = final_tab.columns.difference(train_data.columns)
final_tab = final_tab.drop(columns=columns_to_drop)

In [152]:
print(dgm_oni['ONI_CIRCUITS'].shape)
print(final_tab.shape)
print(train_data.shape)

(2016, 8)
(72, 96)
(685181, 124)


In [149]:
final_tab.head()

,id,dmc,nr_dgm,status,czas_fazy_1,czas_fazy_2,czas_fazy_3,max_predkosc,cisnienie_tloka,cisnienie_koncowe,nachdruck_hub,anguss,temp_pieca,oni_temp_curr_f1,oni_temp_fore_f1,oni_temp_fore_f2,vds_air_pressure,vds_vac_hose1,vds_vac_hose2,vds_vac_tank,vds_vac_valve1,vds_vac_valve2,czas_taktu,rn,id_dmc,flow_1,flow_2,flow_3,flow_4,flow_5,flow_6,flow_7,flow_8,flow_9,flow_10,flow_11,flow_12,flow_13,flow_14,flow_15,flow_16,flow_17,flow_18,flow_19,flow_20,flow_21,flow_22,flow_23,flow_24,flow_25,flow_26,flow_27,flow_28,set_point_1,set_point_2,set_point_3,set_point_4,set_point_5,set_point_6,set_point_7,set_point_8,set_point_9,set_point_10,set_point_11,set_point_12,set_point_13,set_point_14,set_point_15,set_point_16,set_point_17,set_point_18,set_point_19,set_point_20,set_point_21,set_point_22,set_point_23,set_point_24,set_point_25,set_point_26,set_point_27,set_point_28,start_delay_1,start_delay_2,start_delay_3,start_delay_4,start_delay_5,start_delay_6,start_delay_7,start_delay_8,start_delay_9,start_delay_10,start_delay_11,start_delay_12,start_delay_13,start_delay_14,start_delay_15,start_delay_16,start_delay_17,start_delay_18,start_delay_19,start_delay_20,start_delay_21,start_delay_22,start_delay_23,start_delay_24,start_delay_25,start_delay_26,start_delay_27,start_delay_28,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,temp_7,temp_8,temp_9,temp_10,temp_11,temp_12,temp_13,temp_14,temp_15,temp_16,temp_17,temp_18,temp_19,temp_20,temp_21,temp_22,temp_23,temp_24,temp_25,temp_26,temp_27,temp_28,assigment_1_16,assigment_2_16,assigment_3_17,assigment_4_16,assigment_5_17,assigment_6_16,assigment_7_16,assigment_8_16,assigment_9_16,assigment_10_16,assigment_11_16,assigment_12_16,assigment_13_16,assigment_14_16,assigment_15_16,assigment_16_16,assigment_17_16,assigment_18_16,assigment_19_17,assigment_20_16,assigment_21_16,assigment_22_16,assigment_23_16,assigment_24_16,assigment_25_16,assigment_26_16,assigment_27_16,assigment_28_16,working_mode_1_2,working_mode_2_2,working_mode_3_3,working_mode_4_2,working_mode_5_3,working_mode_6_2,working_mode_7_1,working_mode_8_2,working_mode_9_2,working_mode_10_2,working_mode_11_2,working_mode_12_2,working_mode_13_2,working_mode_14_2,working_mode_15_2,working_mode_16_2,working_mode_17_2,working_mode_18_2,working_mode_19_3,working_mode_20_2,working_mode_21_2,working_mode_22_2,working_mode_23_2,working_mode_24_2,working_mode_25_1,working_mode_26_2,working_mode_27_2,working_mode_28_2
0,1474001,231212042730911025738,9,1,1920,88,14.0,5.0,8,277,10,51,0,75.7,75.6,-1.0,5159.288,121.4699,111.68980,114.1204,440.2778,341.6667,85,1,1474001,10.0,7.5,7.0,8.3,8.6,13.8,0.0,9.5,8.6,2.5,10.0,10.3,7.1,9.6,10.1,10.3,9.3,6.4,10.0,11.1,7.8,5.4,8.8,10.1,0.0,11.6,14.1,11.3,10.0,2.0,0.0,15.0,0.0,25.0,0.0,20.0,2.0,15.0,25.0,25.0,2.0,20.0,20.0,25.0,17.0,4.9,0.0,35.0,15.0,25.0,2.0,15.0,0.0,15.0,25.0,15.0,7.0,15.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,12.0,2.0,5.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,21.0,2.0,2.0,2.0,2.0,2.0,98.6,93.9,83.8,85.3,78.2,76.0,73.4,77.6,83.3,72.1,92.1,103.2,77.2,85.8,79.2,77.9,85.5,76.8,77.8,74.4,73.1,90.2,61.8,80.8,73.5,71.9,102.4,75.6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1
1,1474003,231212042930911025739,9,1,1932,87,14.0,5.0,9,278,10,52,0,74.9,75.0,-1.0,5253.038,122.0486,113.54170,115.9722,427.8935,326.9676,85,1,1474003,9.8,7.5,6.9,8.3,8.8,14.0,0.0,9.6,8.6,2.7,10.0,10.3,7.0,9.6,10.3,10.5,9.3,6.4,9.8,11.1,7.8,5.4,8.6,10.1,0.0,11.6,14.3,11.3,10.0,2.0,0.0,15.0,0.0,25.0,0.0,20.0,2.0,15.0,25.0,25.0,2.0,20.0,20.0,25.0,17.0,4.9,0.0,35.0,15.0,25.0,2.0,15.0,0.0,15.0,25.0,15.0,7.0,15.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,12.0,2.0,5.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,21.0,2.0,2.0,2.0,2.0,2.0,98.9,95.4,84.0,85.5,78.5,75.9,73.4,77.9,83.2,72.1,92.3,103.5,75.8,86.1,79.7,78.2,85.8,76.8,78.0,74.2,73.0,89.6,62.0,81.1,74.0,71.9,102.3,75.7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1
2,1474005,2312120430309110257

(2016, 8)
(72, 193)


In [136]:
train_data = load_csv('final_table_before_normalization.csv')

File to read:
c:/Users/DLXPMX8/Desktop/Projekt_AI/meb_process_data_analysis/src/.data/final_table_before_normalization.csv
